# Export of models to torch script

This note is a workaround for the issue described in the notebook *model_creation.ipynb*. To export the models for one of the test cases CB[1-4], set the *case* variable in the last cell, restart the kernel, and execute all cells.

In [1]:
import pandas as pd
import numpy as np
import torch as pt
from helper_module import SimpleMLP, training_loop
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy

cases = ["CB{:1d}".format(i) for i in range(1, 5)]
print("Case names : ", cases)

output_path = "../output/"

# make torch results reproducible and use double precision
pt.set_default_tensor_type(pt.DoubleTensor)
pt.manual_seed(42)
np.random.seed(42)

Case names :  ['CB1', 'CB2', 'CB3', 'CB4']


In [2]:
log_files = {}
for case in cases:
    log_path = output_path + case + "/log_file.pkl"
    log_files[case] = pd.read_pickle(log_path)

In [3]:
model_dict_rv = {
    "n_inputs" : 1,
    "n_outputs" : 1,
    "n_layers" : 2,
    "n_neurons" : 40,
    "activation" : pt.nn.functional.selu,
    "batch_norm" : False
}

scalers_rv = {}
scaled_data_rv = {}

for case in cases:
    scaler = MinMaxScaler()
    data = scaler.fit_transform(log_files[case][["t", "ub_x"]].values)
    scalers_rv[case] = deepcopy(scaler)
    scaled_data_rv[case] = deepcopy(data)

In [4]:
class RiseVelocityModel(pt.nn.Module):
    def __init__(self, model_dict, model_path, scaler_rv):
        super(RiseVelocityModel, self).__init__()
        self._model = SimpleMLP(**model_dict)
        self._model.load_state_dict(pt.load(model_path))
        self._min_t = pt.tensor(scaler_rv.data_min_[0], dtype=pt.float64)
        self._range_t = pt.tensor(scaler_rv.data_range_[0], dtype=pt.float64)
        self._min_rv = pt.tensor(scaler_rv.data_min_[1], dtype=pt.float64)
        self._range_rv = pt.tensor(scaler_rv.data_range_[1], dtype=pt.float64)
        
    def _scale(self, X):
        return (X - self._min_t) / self._range_t
    
    def _inverse_scale(self, y):
        return y * self._range_rv + self._min_rv
    
    def forward(self, X):
        X = self._scale(X)
        return self._inverse_scale(self._model(X))

In [5]:
model_dict_ar = {
    "n_inputs" : 1,
    "n_outputs" : 2,
    "n_layers" : 2,
    "n_neurons" : 40,
    "activation" : pt.nn.functional.selu,
    "batch_norm" : False
}

scalers_ar = {}
scaled_data_ar = {}

for case in cases:
    scaler = MinMaxScaler()
    time = log_files[case].t.values
    half_axes = log_files[case][["db_x", "db_y"]] / 2.0
    data = scaler.fit_transform(np.concatenate((np.expand_dims(time, 1), half_axes), axis=1))
    scalers_ar[case] = deepcopy(scaler)
    scaled_data_ar[case] = deepcopy(data)

In [6]:
class AxesModel(pt.nn.Module):
    def __init__(self, model_dict, model_path, scaler_ar):
        super(AxesModel, self).__init__()
        self._model = SimpleMLP(**model_dict)
        self._model.load_state_dict(pt.load(model_path))
        self._min_t = pt.tensor(scaler_ar.data_min_[0], dtype=pt.float64)
        self._range_t = pt.tensor(scaler_ar.data_range_[0], dtype=pt.float64)
        self._min_axes = pt.tensor(scaler_ar.data_min_[1:], dtype=pt.float64)
        self._range_axes = pt.tensor(scaler_ar.data_range_[1:], dtype=pt.float64)
        
    def _scale(self, X):
        return (X - self._min_t) / self._range_t
    
    def _inverse_scale(self, y):
        return y * self._range_axes + self._min_axes
    
    def forward(self, X):
        X = self._scale(X)
        return self._inverse_scale(self._model(X))

In [7]:
interface_data = {}
for case in cases:
    file_path = output_path + case + "/interface_data_processed.pkl"
    interface_data[case] = pd.read_pickle(file_path)

In [8]:
def ellipsoidal_radius(time, theta, axes_model):
    """Compute the radius of an ellipse.
    
    Parameters
    ----------
    time - array-like: time, unscaled
    theta - array-like: polar angle
    axes_model - Module: model the predict the half-axes
    
    Returns
    -------
    radius - array-like: radius of ellipse; same shape as theta
    """
    axes = axes_model(pt.from_numpy(time).unsqueeze(-1)).detach().numpy()
    a = axes[:, 0]
    b = axes[:, 1]
    return a * b / np.sqrt(np.square(a * np.sin(theta)) + np.square(b * np.cos(theta)))

In [9]:
model_dict_rad = {
    "n_inputs" : 2,
    "n_outputs" : 1,
    "n_layers" : 4,
    "n_neurons" : 40,
    "activation" : pt.nn.functional.selu,
    "batch_norm" : False
}

scalers_tt = {}
scalers_rad = {}
scaled_data_tt = {}
scaled_data_rad = {}

for case in cases:
    model_path = output_path + "/half_axes_{:s}.pt".format(case)
    model = AxesModel(model_dict_ar, model_path, scalers_ar[case])
    rad_ell = ellipsoidal_radius(interface_data[case].t.values, interface_data[case].theta.values, model)
    scaler = MinMaxScaler()
    data = scaler.fit_transform(interface_data[case][["t", "theta"]].values)
    scalers_tt[case] = deepcopy(scaler)
    scaled_data_tt[case] = deepcopy(data)
    scaler = MinMaxScaler()
    rad_proj = interface_data[case].rad.values/rad_ell
    data = scaler.fit_transform(rad_proj[:, np.newaxis])
    scalers_rad[case] = deepcopy(scaler)
    scaled_data_rad[case] = deepcopy(data)

In [10]:
class RadiusModel(pt.nn.Module):
    def __init__(self, model_dict, model_path, axes_model, scaler_tt, scaler_rad):
        super(RadiusModel, self).__init__()
        self._model = SimpleMLP(**model_dict)
        self._model.load_state_dict(pt.load(model_path))
        self._axes_model = axes_model
        self._min_tt = pt.tensor(scaler_tt.data_min_, dtype=pt.float64)
        self._range_tt = pt.tensor(scaler_tt.data_range_, dtype=pt.float64)
        self._min_rad = pt.tensor(scaler_rad.data_min_, dtype=pt.float64)
        self._range_rad = pt.tensor(scaler_rad.data_range_, dtype=pt.float64)
        
    def _scale(self, X):
        return (X - self._min_tt) / self._range_tt
    
    def _inverse_scale(self, y):
        return y * self._range_rad + self._min_rad
    
    def _ellipsoidal_radius(self, X):
        axes = self._axes_model(X[:, 0].unsqueeze(-1))
        re = axes[:, 0] * axes[:, 1] / pt.sqrt(
            pt.square(axes[:, 0] * pt.sin(X[:, 1])) + pt.square(axes[:, 1] * pt.cos(X[:, 1]))
        )
        return re
    
    def forward(self, X):
        rad_ell = self._ellipsoidal_radius(X).squeeze()
        X = self._scale(X)
        X = self._inverse_scale(self._model(X)).squeeze()
        return X * rad_ell

In [11]:
model_dict_tv = {
    "n_inputs" : 2,
    "n_outputs" : 1,
    "n_layers" : 4,
    "n_neurons" : 40,
    "activation" : pt.nn.functional.selu,
    "batch_norm" : False
}

scalers_tv = {}
scaled_data_tv = {}

for case in cases:
    model_path_rv = output_path + "/rise_vel_{:s}.pt".format(case)
    model_rv = RiseVelocityModel(model_dict_rv, model_path_rv, scalers_rv[case])
    rv_model = model_rv(pt.from_numpy(interface_data[case].t.values).unsqueeze(-1)).detach().squeeze().numpy()
    scaler = MinMaxScaler()
    vt_trans = interface_data[case].vel_theta.values / rv_model
    data = scaler.fit_transform(vt_trans[:, np.newaxis])
    scalers_tv[case] = deepcopy(scaler)
    scaled_data_tv[case] = deepcopy(data)

In [12]:
class TangentialVelocityModel(pt.nn.Module):
    def __init__(self, model_dict, model_path, rv_model, scaler_tt, scaler_tv):
        super(TangentialVelocityModel, self).__init__()
        self._model = SimpleMLP(**model_dict)
        self._model.load_state_dict(pt.load(model_path))
        self._rv_model = rv_model
        self._min_tt = pt.tensor(scaler_tt.data_min_, dtype=pt.float64)
        self._range_tt = pt.tensor(scaler_tt.data_range_, dtype=pt.float64)
        self._min_tv = pt.tensor(scaler_tv.data_min_, dtype=pt.float64)
        self._range_tv = pt.tensor(scaler_tv.data_range_, dtype=pt.float64)
        
    def _scale(self, X):
        return (X - self._min_tt) / self._range_tt
    
    def _inverse_scale(self, y):
        return y * self._range_tv + self._min_tv
    
    def _rise_velocity(self, X):
        return self._rv_model(X[:, 0].unsqueeze(-1))
    
    def forward(self, X):
        rv = self._rise_velocity(X).squeeze()
        X = self._scale(X)
        X = self._inverse_scale(self._model(X)).squeeze()
        return X * rv

In [14]:
##################################
# Select the case to export here #
##################################
case = cases[0]
##################################
model_path = output_path + "/rise_vel_{:s}.pt".format(case)
model_rv_case_1 = RiseVelocityModel(model_dict_rv, model_path, scalers_rv[case])
traced_model_rv_case_1 = pt.jit.trace(model_rv_case_1.eval(), pt.ones((1,1)))
traced_model_rv_case_1.save(output_path + "rise_vel_{:s}.ts".format(case))
model_path = output_path + "/tv_{:s}.pt".format(case)
model_tv_case_1 = TangentialVelocityModel(model_dict_tv, model_path, model_rv_case_1, scalers_tt[case], scalers_tv[case])
traced_model_tv_case_1 = pt.jit.trace(model_tv_case_1.eval(), pt.ones((1,2)))
traced_model_tv_case_1.save(output_path + "tv_{:s}.ts".format(case))
model_path = output_path + "/half_axes_{:s}.pt".format(case)
model_axes_case_1 = AxesModel(model_dict_ar, model_path, scalers_ar[case])
model_path = output_path + "/rad_{:s}.pt".format(case)
model_rad_case_1 = RadiusModel(model_dict_rad, model_path, model_axes_case_1, scalers_tt[case], scalers_rad[case])
traced_model_rad_case_1 = pt.jit.trace(model_rad_case_1.eval(), pt.ones((1,2)))
traced_model_rad_case_1.save(output_path + "rad_{:s}.ts".format(case))